In [1]:
import os
import cv2
import numpy as np
import torch
import dlib
from imutils import face_utils
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from torchvision import transforms  
import torch.nn as nn
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2
import albumentations as A
import torch.nn.functional as F
import math
import time
from imutils import face_utils
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
import mediapipe
from BaseModels.resnetModels import BB_model
from BaseModels.efficientnetModels import BB_model
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from scipy.spatial import distance
from BaseModels.unet_model import  UNET
# device = "cpu"


    


In [2]:
def load_model(model_path):
    """Load Regression model

    Args:
        model_path (_str_): _model path_
    

    Returns:
        _torch model_: _RESNET model_
    """

    model = torch.load(model_path,map_location=device)

    model.eval()

    return model

In [3]:

    
# Latest Model
# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Segmentation/Regression_model_1.140251636505127.pth"

# Mix Dataset
# regression_model_path = '/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/Regression_model_1.487574208665777.pth'

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModels/SpheroPipeLine/Resnet_32_IMG_SIZE__32/Regression_model_0.25639680131442016.pth"

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/Trained_Models/Resnet_32_IMG_SIZE__32/Regression_model_0.25639680131442016.pth"

# regression_model_path = "RecentModels/BioGi4eH2head/Regression_model_0.7620949026704394.pth"

# regression_model_path = "RecentModels/SpheroPipeLine/Resnet_32_IMG_SIZE__32/Regression_model_0.25639680131442016.pth"

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModelsonGpu/RecentModels/AllDataset/Regression_model_1.4074174204430023.pth"

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModelsonGpu/Recent_EffcientNetModels/EfficientNet1_6_smoothl1loss_batch_64.pt"


# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModelsonGpu/Recent_EffcientNetModels/EfficientNet1_6_smoothl1loss_batch_256.pt"


# regression_model_path = "RecentModelsonGpu/Recent_EffcientNetModels/Regression_EffcientNet__epoch_500_SmoothL1Loss_summation_batch_256_resize_64_for_gi4e_bioid_h2head_1.52.pt"

# regression_model_path = "RecentModelsonGpu/Recent_EffcientNetModels/EfficientNetWingLossV1.pt"

# regression_model_path = 'ResnetBaseWithMSEMEANwithoughtCoordConv/checkpoint.pt'


regression_model_path = 'HeatMap_Regression/HeatMapRegressionModel.pth'

REGRESSION_MODEL = load_model(
    model_path=regression_model_path)



RESIZE_AMT = 256



In [39]:
x= count_parameters(REGRESSION_MODEL)

In [41]:
x

31043521

In [36]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
# BioID
val_csv_path = "/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/MP2GAZE/AllCoordinatesMp2GazeTest.csv"


val_df = pd.read_csv(val_csv_path)



In [5]:
val_df.head()

,ImageName,Coordinates,DataSet Type
0,"MPIIGaze/Data/Original/p00/day02/0350.jpg""","[576.75131084363, 446.9430824441167, 595.62107...",MP2GAZE
1,"MPIIGaze/Data/Original/p00/day23/0201.jpg""","[629.1027244977723, 418.5422721626321, 649.159...",MP2GAZE
2,"MPIIGaze/Data/Original/p00/day22/0437.jpg""","[519.3837103984533, 379.3424773235636, 551.963...",MP2GAZE
3,"MPIIGaze/Data/Original/p00/day34/0296.jpg""","[563.4499942709884, 374.6517620194357, 591.877...",MP2GAZE
4,"MPIIGaze/Data/Original/p00/day10/0198.jpg""","[659.4630047049818, 463.58724173032533, 685.19...",MP2GAZE


In [6]:
def selectImgDir(datasetType:str):
    
    
    
    
    
    if datasetType == "MP2GAZE":
        return "/home/nipun/Documents/Uni_Malta/Datasets/"
        
        
        
    elif datasetType =="i2head":
        return "/home/nipun/Documents/Uni_Malta/Datasets/"
        
    elif datasetType == "GI4E":
        
        return "/home/nipun/Documents/Uni_Malta/Datasets/gi4e_database/images/"
        
        
        
        
    
    
    

In [7]:
def heatmap2argmax(heatmap, scale=False):
    N, C, H, W = heatmap.shape
    index = heatmap.view(N,C,1,-1).argmax(dim=-1)
    pts = torch.cat([index%W, index//W], dim=2)
    
    if scale:
        scale = torch.Tensor([W,H], device=heatmap.device)
        pts = _scale(pts, scale)
    
    return pts


def _scale(p, s): return 2 * (p / s) - 1


def prediction_image(model,image):
        
        
        model.eval()
        
        
        img = image[:,:,::-1]
        
        
        img = cv2.resize(img,(RESIZE_AMT,RESIZE_AMT))
        
        img = img/255.0
        
        img = torch.tensor(img,dtype=torch.float32).permute(2,0,1)
        
        img = img.unsqueeze(0).to(device)
        
        with torch.no_grad():
                out_coord =heatmap2argmax(model(img).view(-1,1,256,256))
        
        print()
        print(out_coord)
                
                
        
        
        
        img = img.squeeze(0)

        image = transforms.ToPILImage()
        
        
        pred_coord = out_coord.detach().cpu().numpy()[0][0]
        
        return image,pred_coord

In [8]:
def cropeyes(image, inner_array, outer_array):

    arr = {"top_left": [inner_array[0]-5, inner_array[1]-20],
           "bottom_right": [outer_array[0]+5, outer_array[1]+20]}

    return image[arr["top_left"][1]:arr["bottom_right"][1], arr["top_left"][0]:arr["bottom_right"][0]], arr

In [9]:



def rescale_coordinate(coord,original_image,resize_amt):
    
    h,w = original_image.shape[:2]
    coord[0] = int((coord[0]/resize_amt) * w)
    coord[1] = int((coord[1]/resize_amt) * h)
    
    return coord

In [10]:
def scaleCoorinatesToOriginalImage(pred_coords,eye_margin):
    
    # {'top_left': array([385, 214]), 'bottom_right': array([426, 226])}
    
    x1 = eye_margin["top_left"][0] + pred_coords[0]
    y1 = eye_margin["top_left"][1] + pred_coords[1]
    
    return [x1,y1]
    

In [11]:
def calculateEuclideanDistance(coord1,coord2):
    return (((coord1[0])-float(coord2[0]))^2  + (float(coord1[0])-float(coord2[0]))^2) ^ 0.5

In [12]:
IMG_DIR = "/home/nipun/Documents/Uni_Malta/Datasets/"

In [33]:
maximizedNormalizedError = []
times_array = []
for i, (_, rows) in enumerate(val_df.iterrows()):

        bbox = [int(float(x)) for x in rows["Coordinates"][1:-1].split(',')]

        left_inner = bbox[0:2]
        left_center = bbox[2:4]
        left_outer = bbox[4:6]

        right_inner = bbox[6:8]
        right_center = bbox[8:10]
        right_outer = bbox[10:12]

        img_path = os.path.join(IMG_DIR, rows["ImageName"][:-1])
        # #
        img = cv2.imread(img_path)


        IPD = distance.euclidean(left_center, right_center)
        

       

        left_eye, Leye = cropeyes(img, left_inner, left_outer)
        right_eye, Reye = cropeyes(img, right_inner, right_outer)
        
        start_time = datetime.datetime.now()
        _, pred_l_eye = prediction_image(
            model=REGRESSION_MODEL, image=left_eye)
        
        end_time = datetime.datetime.now()
        
        times_array.append((end_time-start_time).total_seconds())

        _, pred_r_eye = prediction_image(
            model=REGRESSION_MODEL, image=right_eye)
        
        
        pred_l_eye = rescale_coordinate(pred_l_eye, left_eye, RESIZE_AMT)

        pred_r_eye = rescale_coordinate(pred_r_eye, right_eye, RESIZE_AMT)
        
        
        pred_l_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            pred_l_eye, Leye)
        pred_r_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            pred_r_eye, Reye)
        
        
        cv2.circle(img, (int(pred_l_eye_toOriginaImage[0]), int(
            pred_l_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)

        cv2.circle(img, (int(pred_r_eye_toOriginaImage[0]), int(
            pred_r_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)
        
        
        cv2.circle(img, (int(left_inner[0]), int(
            left_inner[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(left_center[0]), int(
            left_center[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(left_outer[0]), int(
            left_outer[1])), 1, (0, 0, 255), -1)

        cv2.circle(img, (int(right_inner[0]), int(
            right_inner[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(right_center[0]), int(
            right_center[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(right_outer[0]), int(
            right_outer[1])), 1, (0, 0, 255), -1)
        
         
          
        left_eye_euclidea_distance = distance.euclidean(
            pred_l_eye_toOriginaImage, left_center)

        right_eye_euclidea_distance = distance.euclidean(
            pred_r_eye_toOriginaImage, right_center)
        
        eMax = max(left_eye_euclidea_distance, right_eye_euclidea_distance)/IPD

        print(eMax)

        maximizedNormalizedError.append(eMax)
    
        # break

/tmp/ipykernel_10273/913963836.py:4: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  pts = torch.cat([index%W, index//W], dim=2)



tensor([[[ 95, 112]]], device='cuda:0')

tensor([[[112, 120]]], device='cuda:0')
0.01681064060301542

tensor([[[100, 108]]], device='cuda:0')

tensor([[[110, 110]]], device='cuda:0')
0.01499526786525223

tensor([[[128,  99]]], device='cuda:0')

tensor([[[136, 111]]], device='cuda:0')
0.014134222674039373

tensor([[[105, 119]]], device='cuda:0')

tensor([[[142, 124]]], device='cuda:0')
0.04407683212845996

tensor([[[124, 107]]], device='cuda:0')

tensor([[[136, 107]]], device='cuda:0')
0.015607862099528717

tensor([[[136,  99]]], device='cuda:0')

tensor([[[140, 108]]], device='cuda:0')
0.01563339910144465

tensor([[[126, 119]]], device='cuda:0')

tensor([[[144, 123]]], device='cuda:0')
0.0055828472590842525

tensor([[[136, 106]]], device='cuda:0')

tensor([[[116, 105]]], device='cuda:0')
0.0636665177057444

tensor([[[124, 103]]], device='cuda:0')

tensor([[[140, 105]]], device='cuda:0')
0.0310860829281539

tensor([[[130, 118]]], device='cuda:0')

tensor([[[124, 110]]], device='cuda:0'

In [34]:
def CheckForLess(list1, val):
    
    l1 = []
    for x in list1:
        
        if x <= val:
            l1.append(x)
    return l1

In [35]:
times_array

[0.043262,
 0.028022,
 0.030422,
 0.046971,
 0.028594,
 0.023727,
 0.027655,
 0.026465,
 0.028114,
 0.026544,
 0.026883,
 0.025137,
 0.026246,
 0.027059,
 0.026753,
 0.02587,
 0.026226,
 0.025693,
 0.025826,
 0.026751,
 0.023445,
 0.026438,
 0.03947,
 0.027644,
 0.026444,
 0.024697,
 0.025728,
 0.026153,
 0.026187,
 0.027228,
 0.024485,
 0.026635,
 0.027244,
 0.02536,
 0.026021,
 0.023219,
 0.025985,
 0.026525,
 0.026225,
 0.028133,
 0.025227,
 0.025945,
 0.026264,
 0.027051,
 0.023557,
 0.023789,
 0.026389,
 0.02693,
 0.024372,
 0.027405,
 0.025934,
 0.025696,
 0.027585,
 0.025679,
 0.027138,
 0.025694,
 0.024566,
 0.024992,
 0.026032,
 0.025609,
 0.024113,
 0.026636,
 0.025075,
 0.025723,
 0.026347,
 0.027111,
 0.023689,
 0.026305,
 0.026486,
 0.026191,
 0.027458,
 0.024047,
 0.027768,
 0.025479,
 0.027397,
 0.027315,
 0.025814,
 0.025592,
 0.026707,
 0.02787,
 0.023927,
 0.027581,
 0.025096,
 0.024976,
 0.02567,
 0.026029,
 0.026666,
 0.026585,
 0.027547,
 0.024179,
 0.025194,
 0.02

In [21]:
e_0_25 = CheckForLess(maximizedNormalizedError,0.25)

e_0_05 = CheckForLess(maximizedNormalizedError,0.05)

e_0_1 = CheckForLess(maximizedNormalizedError,0.1)

In [22]:
(len(e_0_25)/len(maximizedNormalizedError)) *100

100.0

In [23]:
(len(e_0_05)/len(maximizedNormalizedError)) * 100

100.0

In [24]:
len(e_0_1)/len((maximizedNormalizedError)) * 100

88.28125

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(img[:,:,::-1])


In [ ]:
plt.imshow(right_eye)

In [15]:
start_time = time.time()

print(start_time)

1713519254.4548721


In [16]:
end_time = time.time()

print(end_time)

1713519265.5901499


In [18]:
end_time - start_time

11.13527774810791

In [19]:
import time

ms = time.time_ns() 

In [26]:
 import datetime

In [24]:
a = datetime.datetime.now()

In [25]:
b = datetime.datetime.now()

In [28]:
delta = b - a

In [29]:
int(delta.total_seconds() * 1000) 

1320

In [30]:
import datetime
a = 
b = datetime.datetime.now()
c = b - a



In [ ]:
img_path =  "/home/nipun/Documents/Uni_Malta/Datasets/columbia_gaze_data_set/Columbia Gaze Data Set/0039/0039_2m_0P_10V_5H.jpg"

frame = cv2.imread(img_path)

plt.imshow(frame[:,:,::-1])

In [32]:
c.total_seconds()

2.4e-05

In [ ]:
  # MediaPipe
shape_array = captureFaceLandmarks(frame)
        

In [ ]:
left_eye, right_eye,Leye,Reye = cropped_image(frame, shape_array)
        

In [ ]:
plt.imshow(left_eye[:,:,::-1])

In [ ]:
plt.imshow(right_eye[:,:,::-1])

In [ ]:
# left = "/home/nipun/Documents/Uni_Malta/Datasets/Center_Regression/MP2GAZE/Images/417_left.png"

# left_eye = cv2.imread(left)

In [ ]:
_,pred_l_eye = prediction_image(model=REGRESSION_MODEL,image=left_eye)
        
        
_,pred_r_eye = prediction_image(model=REGRESSION_MODEL,image=right_eye)


pred_l_eye = rescale_coordinate(pred_l_eye,left_eye,RESIZE_AMT)

pred_r_eye = rescale_coordinate(pred_r_eye,right_eye,RESIZE_AMT)

In [ ]:
cv2.circle(left_eye,(int(pred_l_eye[0]),int(pred_l_eye[1])),1,(0,255,0),-1)
cv2.circle(right_eye,(int(pred_r_eye[0]),int(pred_r_eye[1])),1,(0,255,0),-1)

In [ ]:
plt.imshow(left_eye[:,:,::-1])

In [ ]:
plt.imshow(right_eye[:,:,::-1])
    

# CoordConv

In [ ]:
import numpy as np


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

plt.ion()
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
def generate_data():
    # generate all of the input images
    n_samples = 56 ** 2
    onehots = np.pad(np.eye(n_samples).reshape((n_samples, 1, 56, 56)), ((0,0), (0, 0), (4,4), (4,4)), "constant")
    onehots = torch.from_numpy(onehots).float()
    with torch.no_grad():
        conv = nn.Conv2d(1, 1, kernel_size=9, padding=4, stride=1)
        conv.weight.data.fill_(1)
        conv.bias.data.fill_(0)
        dataset_x = conv(onehots)
    y_i = torch.arange(56).view(56, 1).repeat(1, 56).view(-1, 1)
    y_j = torch.arange(56).repeat(56, 1).view(-1, 1)
    dataset_y = torch.cat((y_i, y_j), dim=1)

    # split the dataset tensor into train, test sets
    sample_order = np.arange(n_samples)
    
    np.random.shuffle(sample_order)
    train_idxes = sample_order[:2352]
    test_idxes = sample_order[2352:]

    train_x = dataset_x[train_idxes]
    train_y = dataset_y[train_idxes] 
    test_x = dataset_x[test_idxes]
    test_y = dataset_y[test_idxes]

    return train_x, train_y, test_x, test_y

In [ ]:
train_x, train_y, test_x, test_y = generate_data()

In [ ]:
train_x.shape

In [ ]:
samples = 8**2
x = np.pad(np.ones((64,64)).reshape(samples,1,8,8), ((2,2), (0, 0), (4,4), (4,4)),"constant")

In [ ]:
x[63]


# array([[[0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 0., 1.]]])
#

In [ ]:
x = np.eye(4)

In [ ]:
x.shape

In [ ]:
x

In [ ]:
torch.arange(8).

In [ ]:
torch.arange(8).view(8,1)

In [ ]:
torch.arange(8).view(8,1).repeat(1,56)

In [ ]:
torch.arange(8).view(8,1)

In [ ]:
torch.arange(8).view(8,1).repeat(1,8)

In [ ]:
train_x, train_y, test_x, test_y = generate_data()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

plt.ion()
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)



def generate_data():
    # generate all of the input images
    n_samples = 56 ** 2
    onehots = np.pad(np.eye(n_samples).reshape((n_samples, 1, 56, 56)), ((0,0), (0, 0), (4,4), (4,4)), "constant")
    onehots = torch.from_numpy(onehots).float()
    with torch.no_grad():
        conv = nn.Conv2d(1, 1, kernel_size=9, padding=4, stride=1)
        conv.weight.data.fill_(1)
        conv.bias.data.fill_(0)
        dataset_x = conv(onehots)
    y_i = torch.arange(56).view(56, 1).repeat(1, 56).view(-1, 1)
    y_j = torch.arange(56).repeat(56, 1).view(-1, 1)
    dataset_y = torch.cat((y_i, y_j), dim=1)

    # split the dataset tensor into train, test sets
    sample_order = np.arange(n_samples)
    
    np.random.shuffle(sample_order)
    train_idxes = sample_order[:2352]
    test_idxes = sample_order[2352:]

    train_x = dataset_x[train_idxes]
    train_y = dataset_y[train_idxes] 
    test_x = dataset_x[test_idxes]
    test_y = dataset_y[test_idxes]

    return train_x, train_y, test_x, test_y


class CoordConv2d(nn.Module):
    
    def __init__(self, device, in_channels, out_channels, kernel_size, padding, stride, input_size):
        super(CoordConv2d, self).__init__()
        self.device = device
        self.cc_xy = self.make_channels(input_size)
        self.conv = nn.Conv2d(in_channels+2, out_channels, kernel_size=kernel_size, padding=padding, stride=stride)

    def make_channels(self, input_size):
        coord_vals = (2 * torch.arange(input_size) / input_size) - 1
        xchannel = coord_vals.repeat((input_size, 1)).unsqueeze(dim=0)
        ychannel = xchannel.permute(0, 2, 1)
        return torch.cat((xchannel.unsqueeze(dim=0), ychannel.unsqueeze(dim=0)), dim=1)

    def forward(self, x):
        n = x.shape[0]
        x = torch.cat((x, self.cc_xy.repeat(n, 1, 1, 1).to(self.device)), dim=1)
        return self.conv(x)



class CNN(nn.Module):

    def __init__(self, device, coordconv=True):
        super(CNN, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=5, padding=2, stride=2)
        self.bn1 = nn.BatchNorm2d(num_features=1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(2)
        # input (N, C, H, W) is (N, 1, 64, 64)
        if coordconv:
            self.conv1 = CoordConv2d(device, 1, 1, kernel_size=1, padding=0, stride=1, input_size=64)
        else:
            self.conv1 = nn.Conv2d(1, 1, kernel_size=1, padding=0, stride=1)
        self.conv2 = nn.Conv2d(1, 2, kernel_size=3, padding=1, stride=2)
        self.conv3 = nn.Conv2d(2, 2, kernel_size=3, padding=1, stride=2)
        self.fc1 = nn.Linear(2 * 4 * 4, 2)
        
    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = self.relu(self.pool(self.bn2(self.conv2(x))))
        x = self.relu(self.pool(self.bn3(self.conv3(x))))
        x = self.relu(self.fc1(x.view(x.size(0), -1)))
        return x


device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_x, train_y, test_x, test_y = generate_data()
train_x, train_y, test_x, test_y = train_x.to(device), train_y.to(device), test_x.to(device), test_y.to(device)
models = [CNN(device=device, coordconv=False).to(device), CNN(device=device, coordconv=True).to(device)]
model_names = ["Standard", "CoordConv"]
n_epochs = 5000
batch_size = 32

loss_history = [[],[]]  # append validation losses as the models train for eventual plotting.

for model_idx, model in enumerate(models):
    optimizer = optim.Adam(model.parameters(), lr=5e-4)
    criterion = nn.L1Loss()
    for epoch_i in range(n_epochs):
        model.train()
        # sample a batch
        sample_idxes = np.random.randint(2352, size=(batch_size))
        pred_y = model(train_x[sample_idxes])
        loss = criterion(pred_y, train_y[sample_idxes])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch_i + 1) % 50 == 0:
            # evaluate the model
            with torch.no_grad():
                model.eval()
                test_loss = criterion(model(test_x), test_y)
            print("Epoch: {}/{}, Model: {}, Train Loss: {:.3f}, Test Loss: {:.3f}".format(epoch_i + 1, n_epochs, model_names[model_idx], loss.item(), test_loss.item()))
            loss_history[model_idx].append(test_loss.item())

    print("---" * 10, "\n")


# plot the validation loss histories
plt.ion()
epoch_x = np.arange(50, n_epochs+1, 50)
plt.plot(epoch_x, loss_history[0], label="Standard", lw=3)
plt.plot(epoch_x, loss_history[1], label="CoordConv", lw=3)
plt.legend()
plt.title("Supervised Regression Error")
plt.xlabel("Epoch")
plt.ylabel("Error")


# Functions to show the CoordConv predictions on input images after training, if desired.
def test_show(sample_idx):
    plt.imshow(test_x[sample_idx, 0].detach().cpu().numpy(), cmap='Reds')
    with torch.no_grad():
        pred_y = model(test_x)[sample_idx]
    print("Predicted: {}, Target: {}".format(pred_y.detach().cpu().numpy(), test_y[sample_idx].detach().cpu().numpy()))

def train_show(sample_idx):
    plt.imshow(train_x[sample_idx, 0].detach().cpu().numpy(), cmap='Reds')
    with torch.no_grad():
        model.eval()
        pred_y = model(train_x)[sample_idx]
    print("Predicted: {}, Target: {}".format(pred_y.detach().cpu().numpy(), train_y[sample_idx].detach().cpu().numpy()))

In [ ]:
x1 = torch.randn(1,2)
y1 = torch.randn(1,2)
gt = torch.randn(1,2)
catX = torch.cat([x1,y1])
catY = torch.cat([gt,gt])

In [ ]:
x1

In [ ]:
import torch.nn as nn

In [ ]:
loss = nn.L1Loss(reduction='mean')

In [ ]:
loss_1 = loss(x1,gt)
loss_2 = loss(y1,gt)

print(loss_1)
print(loss_2)

# tensor([[0.5266, 0.1171]])
# tensor([[0.6186, 0.3922]])

In [ ]:
loss_3 = loss(catX,catY)

print(loss_3)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = np.arange(5)
y = np.arange(5)

In [ ]:
xx,yy = np.meshgrid(x,y)

In [ ]:
xx

In [ ]:
yy

In [ ]:
plt.plot(xx,yy,marker=".",color="b",linestyle="none")
plt.show()

In [ ]:
np.arange(3)*3

In [ ]:
np.arange(3)*2

In [ ]:
h,v = np.meshgrid(np.arange(3)*3,np.arange(3)*2)

In [ ]:
h

In [ ]:
v

In [ ]:
id1 = 0
id2 = 2

In [ ]:
data = np.array([[1,2,3],
                [2,5,2],
                [1,2,1]])

In [ ]:
data[id1,id2]

In [ ]:
h[id1,id2]

In [ ]:
img = np.array([[0,0,0],
       [0,1,0],
       [0,0,0],])


plt.imshow(img)

#y [1,1]

In [ ]:
import torch
tensor = torch.from_numpy(img)

tensor = tensor.unsqueeze(0).unsqueeze(0)

In [ ]:
import torch.nn as nn

In [ ]:
conv1 = CoordConv2d("cpu", 1, 1, kernel_size=1, padding=0, stride=1, input_size=3)

In [ ]:
conv1(tensor)

In [ ]:
conv1(tensor).shape

In [ ]:
# Image Regression with CoordConv



In [ ]:
class CoordConv2d(nn.Module):
    
    def __init__(self, device, in_channels, out_channels, kernel_size, padding, stride, input_size):
        super(CoordConv2d, self).__init__()
        self.device = device
        self.cc_xy = self.make_channels(input_size)
        self.conv = nn.Conv2d(in_channels+2, out_channels, kernel_size=kernel_size, padding=padding, stride=stride)

    def make_channels(self, input_size):
        coord_vals = (2 * torch.arange(input_size) / input_size) - 1
        
       
        xchannel = coord_vals.repeat((input_size, 1)).unsqueeze(dim=0)
        ychannel = xchannel.permute(0, 2, 1)
        return torch.cat((xchannel.unsqueeze(dim=0), ychannel.unsqueeze(dim=0)), dim=1)

    def forward(self, x):
        n = x.shape[0]
        x = torch.cat((x, self.cc_xy.repeat(n, 1, 1, 1).to(self.device)), dim=1)
        return self.conv(x)

In [ ]:
conv1 = CoordConv2d("cpu", 3, 3, kernel_size=1, padding=0, stride=1, input_size=64)
   
            # self.conv1 = nn.Conv2d(1, 1, kernel_size=1, padding=0, stride=1)

In [ ]:
input_tensor = torch.randn(4,3,64,64).to(device)

# conv1(input_tensor).shape

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# models = [CNN(device=device, coordconv=False).to(device), CNN(device=device, coordconv=True).to(device)]

In [ ]:
regression_Model_1 = BB_model(device,coordConv=True).to(device)

regression_Model_2  = BB_model(device,coordConv=False).to(device)

In [ ]:
# regression_Model_1(input_tensor)

In [ ]:
pred_tensor = torch.randn(4,3,64,64).to(device)

In [ ]:
loss_1 = nn.L1Loss(reduction='none')

In [ ]:
loss_1(pred_tensor,input_tensor).sum(1).sum()

In [ ]:
total_loss = 0 
for j,k in zip(input_tensor,pred_tensor):
    
    
    total_loss += loss_1(k,j).sum(1).sum()
    # print(loss_1(k,j).sum(1).sum())

In [ ]:


epoch_x

In [ ]:
len(trainLossModel2)

In [ ]:
trainLossModel1 = [68.28050513016551,
 65.62488816913806,
 59.81700656288549,
 47.68163339715255,
 23.58207727733411,
 9.820071471364875,
 7.343100547790527,
 6.936299600099263,
 6.561544142271343,
 6.248961147509124,
 5.921486227135909,
 5.421173848603901,
 4.664885972675524,
 3.975954532623291,
 3.5830030566767643,
 3.2883990312877454,
 3.0593762523249577,
 2.8676067904422156,
 2.7213477084511206,
 2.547908155541671,
 2.440237747995477,
 2.3724605660689506,
 2.3558950800644722,
 2.3216646345038163,
 2.3052869721462854,
 2.2401248969529806,
 2.3007123846756783,
 2.305938858734934,
 2.1972941411168954,
 2.1649156746111418,
 2.104501667775606,
 2.1308251995789376,
 2.084116032249049,
 2.1654670050269678,
 2.088952616641396,
 2.0328448320689954,
 2.036092237422341,
 2.0027414372092798,
 2.003695845603943,
 1.9912696637605365,
 1.9280576580449154,
 1.9480871087626408,
 2.0456484117006,
 1.9850581758900692,
 1.9068721595563387,
 1.8830207837255377,
 1.908837356065449,
 1.964211476476569,
 1.9530157540973865,
 1.8911270405116833,
 1.8899118461106952,
 1.8800873066249646,
 1.849120296930012,
 1.8649798945376748,
 1.8417292707844783,
 1.8274180073487132,
 1.8325849708757902,
 1.8079830533579777,
 1.8013163867749666,
 1.8247320401041132,
 1.80855215223212,
 1.7555797100067139,
 1.777278605260347,
 1.7460961153632717,
 1.7326714616072805,
 1.7304664348301135,
 1.7664734501587718,
 1.757043951436093,
 1.709508318650095,
 1.7285376034284894,
 1.6577842172823454,
 1.6729020758679038,
 1.7605895180451243,
 1.7548707347167165,
 1.7114581748058921,
 1.6845240655698275,
 1.7117109235964323,
 1.7370394844757884,
 1.714644614018892,
 1.683782602611341,
 1.6904316136711521,
 1.650501433171724,
 1.635898797135604,
 1.6284060415468717,
 1.6087490194722225,
 1.6075890189722966,
 1.601535401846233,
 1.5595416332546033,
 1.6115675976401882,
 1.64955945391404,
 1.5978446257741827,
 1.5724606827685708,
 1.5371948794314736,
 1.5596598198539333,
 1.5207288767162122,
 1.4957340516542132,
 1.5806434656444348,
 1.5309730391753347,
 1.600862415213334,
 1.5464605657677901]


In [ ]:
valLossModel1 =  [68.90546035766602, 61.56637477874756, 952.7806701660156, 810.3438720703125, 15.786891460418701, 13.454748392105103, 8.070767402648926, 6.924545049667358, 6.517513871192932, 6.228425979614258, 6.068917751312256, 5.536641240119934, 4.160773277282715, 3.6340972781181335, 3.5631832480430603, 3.0644643902778625, 2.78153458237648, 2.654276579618454, 2.45757132768631, 2.413040280342102, 2.308493971824646, 2.2595920860767365, 2.2240691781044006, 2.2414967119693756, 2.1230928897857666, 2.119713634252548, 2.120747923851013, 2.1427029967308044, 2.12529793381691, 2.0213115215301514, 2.0390857458114624, 2.1395826637744904, 1.9609893262386322, 2.0884757936000824, 1.9631588757038116, 1.9820888042449951, 1.914979100227356, 1.9358978867530823, 1.9396913945674896, 1.8884586691856384, 1.9031506776809692, 1.8951497077941895, 1.865460753440857, 1.876004308462143, 1.8659241199493408, 1.885868400335312, 1.8572442531585693, 1.8696589767932892, 1.8276720345020294, 1.9082961976528168, 1.85065758228302, 1.860484391450882, 1.824264407157898, 1.833799421787262, 1.9174040257930756, 1.864008367061615, 1.8715886175632477, 1.8427839279174805, 1.8189807832241058, 1.847440630197525, 1.812896192073822, 1.755580484867096, 1.760489672422409, 1.7367846369743347, 1.8441311419010162, 1.7706548273563385, 1.837672084569931, 1.7540681958198547, 1.7603263556957245, 1.7473979592323303, 1.7059321403503418, 1.7229499816894531, 1.8236200511455536, 1.9254466593265533, 1.7679736614227295, 1.7567171454429626, 1.9713301062583923, 1.7706826031208038, 1.7226713001728058, 1.7720273435115814, 1.7138517796993256, 1.7112639248371124, 1.7340607047080994, 1.6640256643295288, 1.7693849503993988, 1.690043866634369, 1.6531054079532623, 1.6595081388950348, 1.8031052947044373, 1.694405734539032, 1.700481116771698, 1.769815742969513, 1.6867660880088806, 1.6367439329624176, 1.689428985118866, 1.757577896118164, 1.707352727651596, 1.7181978821754456, 1.7547268271446228, 1.691870540380478]

In [ ]:
plt.ion()

plt.plot(epoch_x, valLossModel2, label="Standard", lw=3)
plt.plot(epoch_x, valLossModel1, label="CoordConv", lw=3)
plt.legend()
plt.title("Supervised  Validation Regression Error")
plt.xlabel("Epoch")
plt.ylabel("Error")